In [ ]:
import pandas as pd
import numpy as np
import pathlib as pl
import matplotlib.pyplot as plt
import sys
sys.path.append('../dependencies/')
import pyemu
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
wkdir = pl.Path('./MASTER/')

### look at the raw PHI historgrams

In [ ]:
phi = pd.read_csv(wkdir/ 'prior_mc.phi.actual.csv', index_col = 0).T.iloc[5:]
phi_org = phi.copy()
phi.columns = ['phi']

In [ ]:
phi.hist(bins=50)

### rejection sampling - knock out high PHI realizations

In [ ]:
phitoohigh = 8
phi = phi.loc[phi.phi<phitoohigh]
phi.hist(bins=50)

In [ ]:
reals_to_keep = phi.index
reject_reals = list(set(phi_org.index)-set(reals_to_keep)) # we want these to be able to look at the outliers
len(reals_to_keep)

In [ ]:
reject_reals

## let's look first at the observations ensemble

In [ ]:
obs = pd.read_csv(wkdir / 'prior_mc.0.obs.csv', index_col=0).loc[reals_to_keep] # only keep "good" reals

In [ ]:
obs

### parse into Time series and Maximum depletion groups

In [ ]:
ts_obs = obs[[i for i in obs.columns if not i.endswith(':bdpl')]]
bdpl_obs = obs[[i for i in obs.columns if i.endswith(':bdpl')]]


### plot up the time series

In [ ]:
ts_lox = np.unique([i.split('__')[0] for i in ts_obs.columns])

In [ ]:
ts_lox

In [ ]:
for cts in ts_lox:
    tmp = ts_obs[[i for i in ts_obs.columns if cts in i]].T.copy()
    tmp['time'] = [int(i.split('__')[1]) for i in tmp.index]
    tmp.set_index('time', drop=True, inplace=True)
    plt.figure()
    ax = tmp.plot(alpha=.4, c='k', lw=.1,legend=None)
    tmp['base'].plot(c='r')
    ax.set_title(cts)

### histograms for the max depletion obs

In [ ]:
bdpl_obs = bdpl_obs.T

In [ ]:
with PdfPages('base_dep.pdf') as outpdf:
    numpages = int(np.ceil(len(bdpl_obs)/9))
    for i in range(numpages):
        bdpl_obs.loc[bdpl_obs.index[i*9:(i+1)*9]].T.hist(figsize=(8,8))
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

### now let's check out the parameters

In [ ]:
pars = pd.read_csv(wkdir / 'prior_mc.0.par.csv', index_col=0)
pars_keep = pars.loc[reals_to_keep].T
pars_reject = pars.loc[reject_reals].T

In [ ]:
with PdfPages('pars_kept.pdf') as outpdf:
    numpages = int(np.ceil(len(pars_keep)/9))
    for i in range(numpages):
        pars_keep.loc[pars_keep.index[i*9:(i+1)*9]].T.hist(figsize=(8,8))
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

In [ ]:
with PdfPages('pars_reject.pdf') as outpdf:
    numpages = int(np.ceil(len(pars_keep)/9))
    for i in range(numpages):
        pars_reject.loc[pars_reject.index[i*9:(i+1)*9]].T.hist(figsize=(8,8))
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')